<a href="https://colab.research.google.com/github/Bhuvanjeet/Food-Recommendation-Engine/blob/master/food_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Food Recommendation Engine**

To recommend food items based on similarity in 'brand' and 'ingredients'.


**Project Overview:**

**1-Exploratory Data Analysis - EDA**

**2-Vectorization**

**3-Cosine Similarity**

**4-Input and Output**

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link='https://drive.google.com/open?id=1eEM2edL8PyQWuz_LPOKvveRP9nKzGG8M'

In [7]:
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

1eEM2edL8PyQWuz_LPOKvveRP9nKzGG8M


In [0]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('food_items.txt')  

In [0]:
df=pd.read_csv('food_items.txt')

In [11]:
df.head()

,index,brand,categories,ingredients,manufacturer,title,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,0,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery","Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,Simon Fischer Fruit Bttr Prune Lekvar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,McCormick,"Grocery & Gourmet Food,Food,Grocery","Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",McCORMICK GRILL MATES MOLASSES BACON SEASONING...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Jolly Time,"Grocery & Gourmet Food,Grocery","Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Reese's,Jolly Time Popcorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Ziyad,"Grocery & Gourmet Food,grocery",Mechanically hulled seasame seeds.Allergy Info...,Ziyad,Ziyad Tahini Sesame Sauce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",FALSE,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df.drop('index',axis=1,inplace=True)

In [13]:
df.head()

,brand,categories,ingredients,manufacturer,title,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery","Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,Simon Fischer Fruit Bttr Prune Lekvar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,McCormick,"Grocery & Gourmet Food,Food,Grocery","Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",McCORMICK GRILL MATES MOLASSES BACON SEASONING...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jolly Time,"Grocery & Gourmet Food,Grocery","Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Reese's,Jolly Time Popcorn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ziyad,"Grocery & Gourmet Food,grocery",Mechanically hulled seasame seeds.Allergy Info...,Ziyad,Ziyad Tahini Sesame Sauce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",FALSE,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.iloc[:,5:]

,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df.drop(df.iloc[:,5:],axis=1,inplace=True)

In [16]:
df.head()

,brand,categories,ingredients,manufacturer,title
0,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery","Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,Simon Fischer Fruit Bttr Prune Lekvar
1,McCormick,"Grocery & Gourmet Food,Food,Grocery","Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",McCORMICK GRILL MATES MOLASSES BACON SEASONING...
2,Jolly Time,"Grocery & Gourmet Food,Grocery","Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Reese's,Jolly Time Popcorn
3,Ziyad,"Grocery & Gourmet Food,grocery",Mechanically hulled seasame seeds.Allergy Info...,Ziyad,Ziyad Tahini Sesame Sauce
4,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",FALSE,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops


or you could simply have done:

df = df [ ['brand','categories','ingredients','manufacturer','title'] ]

this will automatically drop other columns

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         9114 non-null   object
 1   categories    10000 non-null  object
 2   ingredients   9972 non-null   object
 3   manufacturer  7317 non-null   object
 4   title         9996 non-null   object
dtypes: object(5)
memory usage: 390.8+ KB


In [18]:
df.describe().transpose()

,count,unique,top,freq
brand,9114,3143,Goya Food,77
categories,10000,4285,Grocery,702
ingredients,9972,9290,100% Pure Coffee.,19
manufacturer,7317,2969,"Dr Pepper/Seven Up, Inc",106
title,9996,9911,Yost Gourmet Pops,7


In [19]:
sum(df.duplicated())

13

In [0]:
#data cleaning
#removing duplicates from df
df.drop_duplicates(inplace=True)

In [21]:
sum(df.duplicated())

0

In [22]:
df.isnull().sum()

brand            883
categories         0
ingredients       28
manufacturer    2673
title              4
dtype: int64

In [0]:
#filling null places with ''
df=df.fillna(value='')

In [24]:
df.isnull().sum()

brand           0
categories      0
ingredients     0
manufacturer    0
title           0
dtype: int64

We want to predict 'title' of food item. So from the dataframe, we can see that it depends basically on two features: 'brand' and 'ingredients'.So, we will combine these two columns into 1 column.

In [0]:
df['prepared_data'] = df['brand'] + " " + df['ingredients']

In [26]:
df.head()

,brand,categories,ingredients,manufacturer,title,prepared_data
0,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery","Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,Simon Fischer Fruit Bttr Prune Lekvar,"Simon Fischer Dried Prunes,Water,Corn Syrup,Su..."
1,McCormick,"Grocery & Gourmet Food,Food,Grocery","Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"McCormick Salt,Sugar,Molasses (Refinery Syrup,..."
2,Jolly Time,"Grocery & Gourmet Food,Grocery","Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Reese's,Jolly Time Popcorn,"Jolly Time Salt, Yellow 5 Lake, Tricalcium Pho..."
3,Ziyad,"Grocery & Gourmet Food,grocery",Mechanically hulled seasame seeds.Allergy Info...,Ziyad,Ziyad Tahini Sesame Sauce,Ziyad Mechanically hulled seasame seeds.Allerg...
4,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",FALSE,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,Fla-Vor-Ice FALSE


**Vectorization**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
#converts a collection of text documents to a matrix of token counts

In [0]:
#bow - bag-of-words model
bow_transformer = CountVectorizer() 
vector_matrix = bow_transformer.fit_transform(df['prepared_data'])

In [29]:
vector_matrix.shape   #to get the shape of sparse matrix

(9987, 10217)

**Cosine Similarity**

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
cos_similar = cosine_similarity(vector_matrix) 

**Input**

In [32]:
food_item = input('Enter the title of the food : ')

Enter the title of the food : Fla-Vor-Ice Plus Giant Pops


In [0]:
def search_index(title):
    return df[df['title'] == title].index

In [0]:
index_item=search_index(food_item)

In [35]:
index_item

Int64Index([4], dtype='int64')

In [0]:
#making a list of similar items to the item entered by the user
similar_items = list(enumerate(cos_similar[int(index_item[0])]))

In [39]:
similar_items[:10]      #to display first 10 elements of the list

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 1.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0)]

In [0]:
#sorting in descending order
sorted_similar_items=sorted(similar_items,key=lambda x:x[1],reverse=True)

In [41]:
sorted_similar_items[:10]     #to display first 10 elements of the sorted list

[(4, 1.0),
 (898, 0.09325048082403138),
 (4101, 0.08703882797784893),
 (4243, 0.08703882797784893),
 (1493, 0.08451542547285165),
 (4733, 0.08111071056538127),
 (4047, 0.07980868844676221),
 (2006, 0.07905694150420949),
 (8950, 0.07293249574894728),
 (2904, 0.07071067811865475)]

**Output**

In [0]:
def search_title(index):
    return df['title'][index]

In [43]:
i=0
for item in sorted_similar_items:
    print(search_title(item[0]))
    i=i+1
    if(i>50):     #printing top 50 similar food items
        break

Fla-Vor-Ice Plus Giant Pops
Hurricane Hi Gravity Malt Liquor, 24 Fl. Oz. Can
Perry's Ice Cream Vanilla, 1.5 QT
Perry's Ice Cream Maple Walnut, 1.5 QT
Ice Breakers Coolmint Sugar Free Mints, 1.5 oz, ()
Tropical Punch Premium Syrup
Outshine D/e Fruit Bat 12ct Cocont Pineapple Orng
Sparkling Ice174 Black Raspberry - 17 Fl Oz
Cofresh Spicy Chick Peas 325g
Natural Light174 Beer - 12pk / 12oz Bottles
Mixed Nuts, Unsalted, 11.5 oz (326 g)
Ice Breakers Duo Mints Grape - 1.3oz
Ice BreakersÂ® Ice CubesÂ® Raspberry Sorbet Sugar Free Gum 10 ct Box
Sparkling Ice Peach Lemonade 17oz
Sparkling Ice Mango Lemonade 17oz
Perry's Ice Cream Neapolitan, 1.5 QT
Klondike Krunch Bar 4 oz Slim-A-Bear Nsa Bars, 6 ct
Sparkling Ice Cherry Limeade 17oz
Ice Breakers Ice Cubes Spearmint Sugar-free Gum - 40pc
Mott's for Tots Fruit Punch
Lotte Koala Mini Cookie, Matcha Chocolate, Family Size, 10 Ct
India Chai Tea Spice 2.20 oz
SweetGourmet Arcor Strawberry Buds Bon Bon Filled Hard Candy
Rishi Tea Organic Herbal Tea Caf

So, we have successfully built a food recommendation engine.